In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import undetected_chromedriver as uc


# 啟動瀏覽器工具的選項

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false') # 不載入圖

driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)
# driver = uc.Chrome(options=my_options)


results = []
sresults=set()

alinks = []
salinks=set()
atitle = []
satitle=set()
plink_ptitle = []
setTmp = set()


In [40]:
import re
import json 
import pandas as pd

In [4]:
def links():
    try:
        links = driver.find_elements(By.CSS_SELECTOR, '  article > h2 > a')
    except:
        print('抓不到')
    
    for link in links:
        plink = link.get_attribute('href')
        ptitle = link.get_attribute('innerText')
        setTmp.add(plink)
        # rr = {
        #     ptitle: plink
        # }
        # plink_ptitle.append(rr)
        



In [2]:
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        # offset = driver.execute_script(
        #     'return window.document.documentElement.scrollHeight;'
        # )

        '''
        或是每次只滾動一點距離，
        以免有些網站會在移動長距離後，
        將先前移動當中的元素隱藏

        例如將上方的 script 改成:
        '''
        offset += 2000

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        
        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(1.5)
        links()
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1
            
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        # if offset >= 600:
        #     break

In [5]:
driver.get(url)

In [ ]:
#滾動爬取
for i in range(10):
    scroll()
    sleep(0.3)
    

In [ ]:
print(setTmp)

In [ ]:
#前往連結 但要休息 
for i in range(len(setTmp)):
    driver.get(list(setTmp)[i])
    end_craw()
    sleep(2)
    if (i/20) == 0 :
        print("累了")
        sleep(30)

In [281]:
all_txt=[]
results=[]
all_school=[]
all_comment=[]
all_hearts=[]
sfloor=set()
tmp=set()

In [ ]:
try:
    comment=driver.find_element(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div').get_attribute('innerText')
except:
    comment='None'

comment.replace('\n','-').split('-')

#https://www.dcard.tw/f/tech_job/p/240599860
#這個連結 沒有評論

In [99]:
def scroll_D():
    
    innerHeight = 0
    offset = 0
    count = 0
    limit = 1
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # offset = driver.execute_script(
        #     'return window.document.documentElement.scrollHeight;'
        # )
        sleep(1)
        offset+=800

        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        if offset >= innerHeight:
           count += 1
    

In [ ]:
scroll_D()

In [282]:
results

[]

In [33]:
def end_craw():
    title = driver.find_element(By.CSS_SELECTOR,'div > h1').get_attribute('innerText')
    #內文
    txt = driver.find_element(By.CSS_SELECTOR,'article > div.atm_lo_c0ivcw.atm_le_1ad2xrm.cgmw135 > div > div').get_attribute('innerText').replace('\n','')
    date=driver.find_element(By.CSS_SELECTOR,' div.atm_c8_3rwk2t.atm_9s_1txwivl.atm_h_1h6ojuz.atm_h3_1fwxnve.i1ym7go5 > div:nth-child(2)').get_attribute('innerText')

    HC_total=driver.find_element(By.XPATH,'//*[@id="__next"]/div[2]/div[2]/div/div/div/div/div[5]/div/div/div[1]').get_attribute('innerText')
    HC_totals=HC_total.replace('\n',' ').split(' ')

    try:
        tickets=driver.find_element(By.CSS_SELECTOR,'#__next > div  div:nth-child(4) > div > ul').get_attribute('innerText')
        ticket=tickets.replace('\n',',')
    except:
        print('沒標籤')
        ticket='None'
    result = {
        '文章標題':title,
        '發文內容':txt,
        '發文時間':date,
        '文章標籤':ticket,
        '愛心數量':HC_totals[0],
        '留言數量':HC_totals[-1],
        '內文留言':''
        }
    # tmp.add(result)
    results.append(result)

In [32]:
def test():
    ctxt=driver.find_element(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji').get_attribute('innerText')
    z=ctxt.replace('\n','')
    x=z.replace('回覆','\n')
    c=x.replace('20',',')
    results.append(c)

In [32]:
tmpcomment=set()
results=[]

In [6]:
comments=driver.find_elements(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > div > div > div >span')
for comment in comments:
        commentss=comment.get_attribute('innerText')
        commentsss=commentss.replace('\n','')
        commentssss=[commentsss]
        tmpcomment.update(commentssss)

In [31]:
def test():
    comments=driver.find_elements(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > div > div > div >span')
    for comment in comments:
            commentss=comment.get_attribute('innerText')
            commentsss=commentss.replace('\n','')
            commentssss=[commentsss]
            tmpcomment.update(commentssss)


In [30]:
def btntest():
    btn=driver.find_elements(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > button')

    for i in range(len(btn)):
        try:
            btn[i].click()
            sleep(2)
            test()
        except:
            continue

In [29]:
def finally_scroll():
    
    innerHeight = driver.execute_script(
    'return window.document.documentElement.scrollHeight;'
    )
    innerHeight+=800
    driver.execute_script(f'''
        window.scrollTo({{
            top: {innerHeight}, 
            behavior: 'smooth' 
        }});
    ''')


In [28]:
def f_btntest():
    btn=driver.find_elements(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > button')

    for i in range(len(btn)):
        try:
            bbb=btn[i].get_attribute('innerText')
            pattern = re.compile(r'查看其他 \d+ 則留言+')
            results = pattern.findall(bbb)
            if results == pattern.findall(bbb) :
                btn[i].click()
            sleep(2)
            test()
        except:
            continue

In [27]:
def scroll_D():
    innerHeight = 0
    offset = 0
    count = 0
    limit = 1
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # offset = driver.execute_script(
        #     'return window.document.documentElement.scrollHeight;'
        # )
        sleep(2)

        
        btntest()
        sleep(1)
        test()
        offset+=800
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        if offset >= innerHeight:
           count += 1
           
    

In [38]:
def end_craw():
    title = driver.find_element(
        By.CSS_SELECTOR, 'div > h1').get_attribute('innerText')
    # 內文
    txt = driver.find_element(By.CSS_SELECTOR, 'article > div.atm_lo_c0ivcw.atm_le_1ad2xrm.cgmw135 > div > div').get_attribute(
        'innerText').replace('\n', '')
    date = driver.find_element(
        By.CSS_SELECTOR, ' div.atm_c8_3rwk2t.atm_9s_1txwivl.atm_h_1h6ojuz.atm_h3_1fwxnve.i1ym7go5 > div:nth-child(2)').get_attribute('innerText')

    try:
        HC_total = driver.find_element(
            By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div/div/div/div/div[5]/div/div/div[1]').get_attribute('innerText')
        HC_totals = HC_total.replace('\n', ' ').split(' ')
    except: 
        HC_total=driver.find_element(By.XPATH,'//*[@id="__next"]/div[2]/div[2]/div/div/div/div/div[6]/div/div/div[1]').get_attribute('innerText')
        HC_totals = HC_total.replace('\n', ' ').split(' ')
    try:
        tickets = driver.find_element(
            By.CSS_SELECTOR, '#__next > div  div:nth-child(4) > div > ul').get_attribute('innerText')
        ticket = tickets.replace('\n', ',')
    except:
        print('ticket沒標籤')
        ticket = 'None'
    result = {
        '文章標題': title,
        '發文內容': txt,
        '發文時間': date,
        '文章標籤': ticket,
        '愛心數量': HC_totals[0],
        '留言數量': HC_totals[-1],
        '內文留言': ''
    }
    # tmp.add(result)
    results.append(result)



def craw_1():
    
    try:
        WebDriverWait(driver, 3).until(
                            EC.presence_of_element_located(
                                (By.CSS_SELECTOR, '#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > div > div > div >span')))
        comments = driver.find_elements(
            By.CSS_SELECTOR, '#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > div > div > div >span')
        for comment in comments:
            commentss = comment.get_attribute('innerText')
            commentsss = commentss.replace('\n', '')
            commentssss = [commentsss]
            tmpcomment.update(commentssss)
    except:
        commentssss='None'
        tmpcomment.update(commentssss)


def btntest():
    btn = driver.find_elements(
        By.CSS_SELECTOR, '#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > button')

    for i in range(len(btn)):
        try:
            btn[i].click()
            sleep(2)
            craw_1()
        except:
            continue


def finally_scroll():

    innerHeight = driver.execute_script(
        'return window.document.documentElement.scrollHeight;'
    )
    innerHeight += 800
    driver.execute_script(f'''
        window.scrollTo({{
            top: {innerHeight}, 
            behavior: 'smooth' 
        }});
    ''')


def f_btntest():
    btn = driver.find_elements(
        By.CSS_SELECTOR, '#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > button')

    for i in range(len(btn)):
        try:
            btn[i].click()
            sleep(2)
            craw_1()
            finally_scroll()
        except:
            continue


def scroll_D():
    innerHeight = 0
    offset = 0
    count = 0
    limit = 1

    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # offset = driver.execute_script(
        #     'return window.document.documentElement.scrollHeight;'
        # )
        sleep(2)

        btntest()
        sleep(1)
        craw_1()
        offset += 800
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        if offset >= innerHeight:
            count += 1


In [41]:
if __name__ == '__main__' :
    end_craw()
    scroll_D()
    btntest()
    f_btntest()
    finally_scroll()
    craw_1()
    for i in tmpcomment:
        i.replace('','None')
        results[0]['內文留言']=results[0]['內文留言']+i

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x0030ACD3+2075859]
	Ordinal0 [0x0029EE61+1633889]
	Ordinal0 [0x0019B7BD+571325]
	Ordinal0 [0x001CAC2F+764975]
	Ordinal0 [0x001CAE1B+765467]
	Ordinal0 [0x001FD0F2+970994]
	Ordinal0 [0x001E7364+881508]
	Ordinal0 [0x001FB56A+963946]
	Ordinal0 [0x001E7136+880950]
	Ordinal0 [0x001BFEFD+720637]
	Ordinal0 [0x001C0F3F+724799]
	GetHandleVerifier [0x005BEED2+2769538]
	GetHandleVerifier [0x005B0D95+2711877]
	GetHandleVerifier [0x0039A03A+521194]
	GetHandleVerifier [0x00398DA0+516432]
	Ordinal0 [0x002A682C+1665068]
	Ordinal0 [0x002AB128+1683752]
	Ordinal0 [0x002AB215+1683989]
	Ordinal0 [0x002B6484+1729668]
	BaseThreadInitThunk [0x75A0FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77317B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77317B2E+238]


In [39]:
results

[{'文章標題': '某代工廠 NB PCC Dell team 、 Lenovo team 選擇',
  '發文內容': '想問問板上各大前輩，對於這間公司PCC一職，以上兩個部門的選擇。先前在PTT爬文有看到大家說DELL很操，但是適合新人練功的好地方；但Lenovo NB這塊沒有看到什麼分享，想詢問看看這家的Lenovo Nb team如何，以及在這兩team間會如何做選擇，謝謝大家🥹',
  '發文時間': '4月6日 23:30',
  '文章標籤': 'Lenovo,Dell,NB',
  '愛心數量': '2',
  '留言數量': '26',
  '內文留言': ''},
 {'文章標題': '金融時報：美晶片出口管制將中企打回石器時代',
  '發文內容': '無法取得網頁資訊www.cna.com.tw（中央社記者陳韻聿倫敦9日專電）美國商務部7日宣布新一波對中國的晶片和半導體技術出口管制，有中國金融業高層稱這是將中方相關產業打回「石器時代」，部份分析家則認為，台灣相關企業或許能漁翁得利。英國「金融時報」（Financial Times）今天報導，約在兩年前，華為首次遭美國嚴厲制裁重擊後營收大減，在市場失去領導地位，創辦人任正非甚至曾告知屬下公司瀕臨生存危機。不過，美方最新一波管制衝擊的是整個中國半導體產業，而不僅是個別企業。現在，使用美國技術生產並應用於人工智慧、高效能運算和超級電腦的半導體產品，僅能在取得政府核發的出口許可後，銷往中國，但取得相關許可十分困難。此外，除非事先取得特別核可，美國政府禁止美國公民或實體與中國的晶片製造商合作。根據7日宣布的措施，美方也嚴加管制晶片製造所需工具和技術的出口，以限制中國發展自己的設備。中國金融集團「華興資本」董事總經理Szeho Ng指出，中國企業「基本上正退回石器時代」，而這還只是「委婉的說法」。顧問公司「歐布萊特石橋集團」（Albright Stonebridge Group）的中國和科技專家崔歐洛（Paul Triolo）說，在這場「席捲半導體及相關產業的海嘯劇變」中，會有「很多輸家」。不過，他提到，儘管需要一段時間才能釐清產業受衝擊程度的完整圖像，但最少可以確定的是，中國和美國的科技創新速度都會放緩，且美國的消費者和企業最終得為此付出數億、甚至數十億美元成本。美國的新管制措施也影響第三國家晶片製

In [34]:
def com_app():
    for i in tmpcomment:
        i.replace('','None')
        results[0]['內文留言']=results[0]['內文留言']+i


In [33]:
results[0]['內文留言']

''

In [39]:
end_craw()

In [41]:
start=1
end=0

In [57]:
print((f'{folderPath_w}{filename_innerText}.json'))

C:/Users/student/python_web_scraping-master/Dcard/Dcard_V1.json


In [ ]:

df = pd.read_json(f'{folderPath_w}{filename_innerText}.json')
df.to_csv(f'{folderPath_w}{filename_innerText}'+'.csv',
            index=None, encoding="utf_8_sig")

In [26]:
def btntest():
    btn=driver.find_elements(By.CSS_SELECTOR,'#dcard-comment-anchor > div > div > div.atm_l8_1077ktj.c34rbji > section > div > div > div > div > div > button')

    for i in range(len(btn)):
        try:
            pattern = re.compile(r'查看其他 \d+ 則留言+')
            results = pattern.findall(bbb)
            if results == pattern.findall(bbb) :
                print(True)
            btn[i].click()
            sleep(2)
            test()
        except:
            continue

查看其他 1 則留言
隱藏留言


In [55]:
HC_totals='None','None'
HC_totals[0]

'None'

In [59]:
def end_craw():
    title = driver.find_element(
        By.CSS_SELECTOR, 'div > h1').get_attribute('innerText')
    # 內文
    txt = driver.find_element(By.CSS_SELECTOR, 'article > div.atm_lo_c0ivcw.atm_le_1ad2xrm.cgmw135 > div > div').get_attribute(
        'innerText').replace('\n', '')
    date = driver.find_element(
        By.CSS_SELECTOR, ' div.atm_c8_3rwk2t.atm_9s_1txwivl.atm_h_1h6ojuz.atm_h3_1fwxnve.i1ym7go5 > div:nth-child(2)').get_attribute('innerText')

    try:
        HC_total = driver.find_element(
            By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div/div/div/div/div[5]/div/div/div[1]').get_attribute('innerText')
        HC_totals = HC_total.replace('\n', ' ').split(' ')
    except: 
        HC_total=driver.find_element(By.XPATH,'//*[@id="__next"]/div[2]/div[2]/div/div/div/div/div[6]/div/div/div[1]').get_attribute('innerText')
        HC_totals = HC_total.replace('\n', ' ').split(' ')
    try:
        tickets = driver.find_element(
            By.CSS_SELECTOR, '#__next > div  div:nth-child(4) > div > ul').get_attribute('innerText')
        ticket = tickets.replace('\n', ',')
    except:
        print('ticket沒標籤')
        ticket = 'None'
    result = {
        '文章標題': title,
        '發文內容': txt,
        '發文時間': date,
        '文章標籤': ticket,
        '愛心數量': HC_totals[0],
        '留言數量': HC_totals[-1],
        '內文留言': ''
    }
    # tmp.add(result)
    results.append(result)


In [15]:
results={'hellO':'HE'}

In [18]:
import json
import pandas as pd

In [ ]:

folderPath_w = 'C:/Users/student/python_web_scraping-master/Dcard/'
filename_innerText = 'Dcard_V1'

In [28]:
folderPath_w = 'C:/Selenium/dcard/'
filename_innerText = 'Dcard_V1'

In [29]:
def savejson():
    with open(f'{folderPath_w}{filename_innerText}'+'.json', "w", encoding='utf-8') as file:
        file.write(json.dumps(
            results, ensure_ascii=False, indent=4))
    print("檔案", filename_innerText, "存好了")

    df = pd.read_json(f'{folderPath_w}{filename_innerText}.json')
    df.to_csv(f'{folderPath_w}{filename_innerText}'+'.csv',
              index=None, encoding="utf_8_sig")


In [30]:
savejson()

檔案 Dcard_V1 存好了
